# Transportation:  array-valued data and decisions

In [ ]:
import pandas as pd
import numpy as np

import pyomo.environ as pe
import pyomo_helpers as helpers

%matplotlib inline

### Read and convert data

Let's make our lives easier and manipulate the spreadsheet to make it really easy to read the data in!

In [ ]:
# Read in the data first, extracting on the parameters needed
cost = pd.read_excel('transportation-lp.xlsx', sheet_name='cost', index_col=0)
demand = pd.read_excel('transportation-lp.xlsx', sheet_name='demand', index_col=0)
capacity = pd.read_excel('transportation-lp.xlsx', sheet_name='capacity', index_col=0)

### Define model

Let's create two indexes called `plant` and `warehouse` and use them to define the decision variable `model.x`.

In [ ]:
model = pe.AbstractModel()

num_warehouse = demand.shape[0]
num_plant = capacity.shape[0]

model.warehouse_i = pe.RangeSet(1, num_warehouse)
model.plant_i = pe.RangeSet(1, num_plant)

model.x = pe.Var(model.plant_i, model.warehouse_i, domain=pe.NonNegativeReals)

Next we'll use the `helpers` functions to create dictionaries that Pyomo can use for the parameters.

In [ ]:
# Create the dictionaries
cost_dict = helpers.convert_2d_array_to_dict(cost)
demand_dict = helpers.convert_1d_array_to_dict(demand)
capacity_dict = helpers.convert_1d_array_to_dict(capacity)

# Create the model parameter objects
model.cost = pe.Param(model.plant_i, model.warehouse_i, initialize=cost_dict)
model.demand = pe.Param(model.warehouse_i, initialize=demand_dict)
model.capacity = pe.Param(model.plant_i, initialize=capacity_dict)

Our constraints here do not involve sumproducts, but rather row and column sums.  

Finally, our objective is a sumproduct over ALL decisions and costs.

### Solve and analyze

In [ ]:
instance, obj, solution = helpers.solve(model)

In [ ]:
# This is how you get the objective... here "obj" is what you named the objective object
# when you created the model.
instance.obj.expr()

In [ ]:
# The dataframe is stored as a single column (i.e., a flattened array),
# but we want to display it as an array like in the spreadsheet.
solution.shape

In [ ]:
# Here is an example of how to reshape it into an array and give it appropriate row/col labels.
solution = pd.DataFrame(solution.values.reshape(num_plant, num_warehouse),
                        index=cost.index, 
                        columns=cost.columns)
solution

Let's visualize the solution with a heatmap from the seaborn package.

In [ ]:
import seaborn as sns

_ = sns.heatmap(solution, annot=True)